In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sklearn
sklearn.__version__

'0.20.2'

In [3]:
import pandas as pd
import pickle as pkl

In [4]:
%run utils.ipynb
%run targets.ipynb

In [5]:
file = 'ETHBTC_1m_1519496760000_1549568220000.csv'

In [15]:
# Get Data
df = pd.read_csv('../data/'+file)
df.shape
df.head(10)

(500000, 12)

,OPEN_TIME,OPEN,HIGH,LOW,CLOSE,VOLUME,CLOSE_TIME,QUOTE_VOL,NUM_TRADES,TAKE_BASE_VOL,TAKE_QUOTE_VOL,EXTRA
0,1519496760000,0.085541,0.085570,0.085407,0.085473,27.714,1519496819999,2.370345,192,15.308,1.309733,0
1,1519496820000,0.085473,0.085570,0.085448,0.085449,20.128,1519496879999,1.720609,122,6.579,0.562654,0
2,1519496880000,0.085448,0.085569,0.085400,0.085538,45.510,1519496939999,3.890176,209,19.426,1.660960,0
3,1519496940000,0.085538,0.085546,0.085402,0.085540,22.231,1519496999999,1.900513,105,14.320,1.224483,0
4,1519497000000,0.085539,0.085552,0.085464,0.085464,15.243,1519497059999,1.303412,117,8.373,0.716219,0
5,1519497060000,0.085466,0.085555,0.085464,0.085545,31.895,1519497119999,2.727413,131,19.237,1.645547,0
6,1519497120000,0.085545,0.085545,0.085464,0.085539,13.325,1519497179999,1.139346,78,7.311,0.625352,0
7,1519497180000,0.085539,0.085555,0.085414,0.085517,28.202,1519497239999,2.411093,195,14.409,1.232346,0
8,1519497240000,0.085460,0.085545,0.085419,0.085534,15.302,1519497299999,1.307959,134,7.216,0.617035,0
9,1519497300000,0.085440,0.085545,0.085440,0.085533,13.730,1519497359999,1.173580,105,6.186,0.529001,0


In [16]:
data = df.loc[:, ['OPEN_TIME', 'CLOSE_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.shape
data[:5]

(500000, 6)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464


In [17]:
# Get Target
%run targets.ipynb
target = get_target_1('../data/'+file)
target.shape
target.head()

(499999, 2)

,CLOSE_TIME,TARGET
0,1519496819999,1.0
1,1519496879999,0.0
2,1519496939999,0.0
3,1519496999999,1.0
4,1519497059999,0.0


In [18]:
data = pd.merge(data, target, on='CLOSE_TIME')
data.shape
data.head()

(499999, 7)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE,TARGET
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473,1.0
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449,0.0
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538,0.0
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540,1.0
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464,0.0


In [19]:
TR_split, CV_split, TT_split = TR_CV_TT_split(data, 0.6, 0.2, 0.2)

TR_split.shape
CV_split.shape
TT_split.shape

TR_split.head()

(299999, 7)

(100000, 7)

(100000, 7)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE,TARGET
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473,1.0
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449,0.0
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538,0.0
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540,1.0
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464,0.0


In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
window = 60
def data_transformer(id, data, target, retrain=False):
    global scaler
    global window
    
    if retrain==True:
        scaler = scaler.fit(data)
        
    data = scaler.transform(data)
    return get_multiple_ts_dataset(id, data, target, tr_win=window, tt_win=0, point_target=True)

In [21]:
TR_id     = TR_split.iloc[:, :2  ].values
TR_data   = TR_split.iloc[:, 2:-1].values
TR_target = TR_split.iloc[:, -1: ].values

CV_id     = CV_split.iloc[:, :2  ].values
CV_data   = CV_split.iloc[:, 2:-1].values
CV_target = CV_split.iloc[:, -1: ].values

TT_id     = TT_split.iloc[:, :2  ].values
TT_data   = TT_split.iloc[:, 2:-1].values
TT_target = TT_split.iloc[:, -1: ].values

TR_id.shape
TR_data.shape
TR_target.shape

CV_id.shape
CV_data.shape
CV_target.shape

TT_id.shape
TT_data.shape
TT_target.shape

(299999, 2)

(299999, 4)

(299999, 1)

(100000, 2)

(100000, 4)

(100000, 1)

(100000, 2)

(100000, 4)

(100000, 1)

In [22]:
TR_id, TR_X, TR_y = data_transformer(TR_id, TR_data, TR_target, retrain=True)
CV_id, CV_X, CV_y = data_transformer(CV_id, CV_data, CV_target, retrain=False)
TT_id, TT_X, TT_y = data_transformer(TT_id, TT_data, TT_target, retrain=False)

TR_id.shape
TR_X.shape
TR_y.shape

CV_id.shape
CV_X.shape
CV_y.shape

TT_id.shape
TT_X.shape
TT_y.shape

(299940, 2)

(299940, 240)

(299940, 1)

(99941, 2)

(99941, 240)

(99941, 1)

(99941, 2)

(99941, 240)

(99941, 1)

In [30]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 2, 2, 2, 2]
metrics.completeness_score(labels_true, labels_pred) 

0.5

In [47]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(
    n_neighbors=30, 
    weights='uniform', 
    algorithm='auto', 
    leaf_size=30, 
    p=1, 
    metric='minkowski', 
    metric_params=None, 
    n_jobs=-1
)

model = model.fit(TR_X, TR_y)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [35]:
TR_y_pred = model.predict_proba(TR_X)[:, 1]
CV_y_pred = model.predict_proba(CV_X)[:, 1]
TT_y_pred = model.predict_proba(TT_X)[:, 1]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(CV_metrics, name='CV'))
res = res.append(pd.Series(TT_metrics, name='TT'))

res

,acc,auc,f1,log_loss
TR,0.999997,0.999997,0.999997,0.000115
CV,0.497363,0.498190,0.494068,17.360669
TT,0.492761,0.496738,0.541853,17.519682


In [37]:
TR_y_pred = model.predict_proba(TR_X)[:, 1]
CV_y_pred = model.predict_proba(CV_X)[:, 1]
TT_y_pred = model.predict_proba(TT_X)[:, 1]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(CV_metrics, name='CV'))
res = res.append(pd.Series(TT_metrics, name='TT'))

res

,acc,auc,f1,log_loss
TR,0.999997,0.999997,0.999997,0.000115
CV,0.498584,0.499537,0.496605,17.318507
TT,0.498864,0.499846,0.505909,17.308832


In [39]:
TR_y_pred = model.predict_proba(TR_X)[:, 1]
CV_y_pred = model.predict_proba(CV_X)[:, 1]
TT_y_pred = model.predict_proba(TT_X)[:, 1]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(CV_metrics, name='CV'))
res = res.append(pd.Series(TT_metrics, name='TT'))

res

,acc,auc,f1,log_loss
TR,0.999997,0.999997,0.999997,0.000115
CV,0.500255,0.500055,0.485808,17.260783
TT,0.506629,0.504387,0.460372,17.040610


In [48]:
TR_y_pred = model.predict_proba(TR_X)[:, 1]
CV_y_pred = model.predict_proba(CV_X)[:, 1]
TT_y_pred = model.predict_proba(TT_X)[:, 1]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(CV_metrics, name='CV'))
res = res.append(pd.Series(TT_metrics, name='TT'))

res

,acc,auc,f1,log_loss
TR,0.577819,0.577254,0.561441,14.581789
CV,0.495993,0.498235,0.507369,17.408028
TT,0.503247,0.502611,0.484733,17.157441
